In [1]:
from dotenv import load_dotenv
from openai import OpenAI
load_dotenv()
import os
import json
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"), organization='org-LB0f6h3G5argFODGs1DOSLNn')

### Get Decription for Each Agent

In [26]:
"""
Ask gpt to generate profile summary of each persona.
"""
def gpt_summarize_profile(text, client, names):
    schema = {
        "pseudonym": "Isabella",
        "description": "Isabella is a 25-year-old woman who loves to cook and bake. She works at a bakery on the corner of the street. She is a perfectionist and always strives to make things perfect. On her days off, she goes to the park to enjoy the sunshine."
    }

    prompt = f"""
    Given the input interview script, generate a pseudonym for the person based on the profile (consider cultural, ethnic diversity and background). Also, provide a general profile description including basic information about the person, their personality, hobbies, etc. Output in Json format in the following schema.

    For example: 
    
    {{
        "pseudonym": "Isabella",
        "description": "Isabella is a 25-year-old woman who loves to cook and bake. She works at a bakery on the corner of the street. She is a perfectionist and always strives to make things perfect. On her days off, she goes to the park to enjoy the sunshine."
    }}

    Avoid these names that have already been selected: {names} Avoid them and choose something else!

    Input: {text}
    Output format: JSON object matching this structure: {json.dumps(schema, indent=2)}
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": "Provide output in valid JSON with this schema:" + json.dumps(schema)},
            {"role": "user", "content": prompt}
        ],
        max_tokens=300,
        temperature=0.3,
    )
    
    data =  json.loads(response.choices[0].message.content)
    return data


In [27]:
"""
Create an agent list with all agent in AgentBank
"""
import os
import glob

folder_path = 'AgentBank'
output_file_path = "sunetid_to_pseudonyms.txt"
txt_files = glob.glob(os.path.join(folder_path, '*.txt'))

agent_list = []
names = "Lian, Mateo, Amy, "
with open(output_file_path, 'w') as output_file:
    for file_path in txt_files:
        file_name = os.path.basename(file_path)
    
        with open(file_path, 'r') as file:
            text = file.read()  
            data = gpt_summarize_profile(text, client, names)
            name = data["pseudonym"]
            persona = data["description"]
            names = names + name + ", "

            agent_profile = {
                "name": name,
                "persona": persona
            }
            print("current agent profile", agent_profile)
            print("existing names", names)
            print("-" * 20)
            agent_list.append(agent_profile)
            output_file.write(f"{file_name},{name}\n")

print(agent_list)

current agent profile {'name': 'Julian', 'persona': "Julian is a 20-year-old man studying economics at Stanford University, with a minor in history and plans to pursue a master's in computer science. He has a passion for debate and enjoys researching various topics, which has led him to develop a wealth of esoteric knowledge. Julian values strong moral convictions in others and appreciates friendships that have helped him through difficult times. He lives with his girlfriend, enjoys large breakfasts, and relies on coffee to stay focused. Despite a busy schedule filled with classes and debate tournaments, he strives to actualize himself and contribute positively to the world."}
existing names Lian, Mateo, Amy, Julian, 
--------------------
current agent profile {'name': 'Sakura', 'persona': 'Sakura is a 20-year-old woman majoring in computer science with a concentration in human-computer interaction. She has a passion for debate and is currently the vice president of her debate team. Sa

In [ ]:
{
  "name": "Isabella",
  "description": "Isabella is a 25 year old woman who loves to cook and bake. She works at a bakery which is on the corner of the street. She is a perfectionist and always strives to make things perfect. On her days off, she goes to the park to enjoy the sunshine."
}

In [28]:
print(agent_list)

[{'name': 'Julian', 'persona': "Julian is a 20-year-old man studying economics at Stanford University, with a minor in history and plans to pursue a master's in computer science. He has a passion for debate and enjoys researching various topics, which has led him to develop a wealth of esoteric knowledge. Julian values strong moral convictions in others and appreciates friendships that have helped him through difficult times. He lives with his girlfriend, enjoys large breakfasts, and relies on coffee to stay focused. Despite a busy schedule filled with classes and debate tournaments, he strives to actualize himself and contribute positively to the world."}, {'name': 'Sakura', 'persona': 'Sakura is a 20-year-old woman majoring in computer science with a concentration in human-computer interaction. She has a passion for debate and is currently the vice president of her debate team. Sakura is a fun-loving individual who values joy and competence in others. She enjoys cooking, playing vide

### Create Agent Folders

In [3]:
questions = [
    "name",
    "age group",
    "gender",
    "grow up location",
    "Which of the following activities do you spend the most time on?",
    "Which value is most important to you?",
    "How would you describe the personality of your closest friend or family member?",
    "How do you typically act in unfamiliar social contexts?",
    "If you had infinite money, how would you spend most of your time?",
    "What is your favorite hobby?",
    "political affiliation",
    "How many places have you lived in?",
    "What is most important to you in social relationships?",
    "How would you describe your childhood?",
    "MBTI type",
    "primary goal for the next 5 years",
    "What do you fear the most?",
    "Have you experienced any childhood trauma that affects you today?",
    "How often do you experience intrusive thoughts?",
    "What has been one of the most meaningful events in your life?",
    "Have you experienced tension growing up between different cultural norms?",
    "When solving a difficult situation, what is your primary approach?",
    "How would you describe your religious or spiritual beliefs?",
    "What is your most prized possession?",
    "What is your biggest career aspiration?",
    "How do you solve difficult situations?",
    "What trait do you value most in friends?",
    "What would you do with $100?",
    "What is your household income level?",
    "How do you identify your ethnicity?",
    "What is your primary language spoken at home?"
]

In [4]:
print(len(questions))

30


In [6]:
import pandas as pd

info = pd.read_csv("charlotte/merged_file.csv")
info.head()

,sunet,name,q1,q2,q3,q4,q5,q6,q7,q8,...,q21,q22,q23,q24,q25,q26,q27,q28,q29,q30
0,bwthomas,Julian,18-24,male,urban,work,integrity,analytical,reserved,working on personal projects,...,logical analysis,athiest,tech gadget,becoming a leader in my field,analyzing all options,intelligence,buy something special,"60,000-100,000",white/caucasian,english
1,changamy,Sakura,18-24,female,urban,hobbies,ambition,analytical,awkward,traveling,...,intuition,atheist,tech gadget,helping others,taking immediate action,intelligence,save it,"$30,000 - $60,000",asian,chinese
2,riyadule,Riya,18-24,female,urban,work,compassion,outgoing,friendly,travelling,...,intuition,spiritual but not religious,home,achieving work-life balance,seeking advice,humor,save it,"over $150,000",asian,english
3,huanxing,Kai,18-24,male,urban,work,compassion,introverted,awkward,travelling,...,asking for help,agnostic,tech gadget,achieving work-life balance,analyzing all options,empathy,buy something special,"$60,000-$100,000",asian,chinese
4,wpo,Jinwoo,25-34,male,rural,work,ambition,introverted,reserved,working on personal projects,...,intuition,agnostic,jewelry,financial independence,analyzing all options,honesty,nothing,"over $150,000",white/caucasian,english


In [8]:
questions = [
    "name",
    "age group",
    "gender",
    "grow up location",
    "Which of the following activities do you spend the most time on?",
    "Which value is most important to you?",
    "How would you describe the personality of your closest friend or family member?",
    "How do you typically act in unfamiliar social contexts?",
    "If you had infinite money, how would you spend most of your time?",
    "What is your favorite hobby?",
    "political affiliation",
    "How many places have you lived in?",
    "What is most important to you in social relationships?",
    "How would you describe your childhood?",
    "MBTI type",
    "primary goal for the next 5 years",
    "What do you fear the most?",
    "Have you experienced any childhood trauma that affects you today?",
    "How often do you experience intrusive thoughts?",
    "What has been one of the most meaningful events in your life?",
    "Have you experienced tension growing up between different cultural norms?",
    "When solving a difficult situation, what is your primary approach?",
    "How would you describe your religious or spiritual beliefs?",
    "What is your most prized possession?",
    "What is your biggest career aspiration?",
    "How do you solve difficult situations?",
    "What trait do you value most in friends?",
    "What would you do with $100?",
    "What is your household income level?",
    "How do you identify your ethnicity?",
    "What is your primary language spoken at home?"
]


base_path = "charlotte/agent_bank/populations/SyntheticCS222_Base/"
for index, row in info.iterrows():
# get the name
    name = row['name']
    base_folder_path = os.path.join(base_path, name) 
    memory_stream_path = os.path.join(base_folder_path, "memory_stream")

    os.makedirs(memory_stream_path, exist_ok=True)

    meta_data = {
        "population": "SyntheticCS222_Base",
        "id": name.lower().replace(" ", "_"),
        "forked_population": "SyntheticCS222_Base",
        "forked_id": name.lower().replace(" ", "_")
    }

    # create meta

    meta_file_path = os.path.join(base_folder_path, "meta.json")  # Create meta.json next to memory_stream
    with open(meta_file_path, 'w') as meta_file:
        json.dump(meta_data, meta_file, indent=4)
    
    # create scratch
    scratch_data = {questions[i]: row[i+1] for i in range(len(questions))}
    
    scratch_file_path = os.path.join(base_folder_path, "scratch.json")  # Create scratch.json next to memory_stream
    with open(scratch_file_path, 'w') as scratch_file:
        json.dump(scratch_data, scratch_file, indent=4)

    # Create the embeddings.json file (inside the memory_stream folder)
    embeddings_data = {}  # Empty object for embeddings.json
    embeddings_file_path = os.path.join(memory_stream_path, "embeddings.json")
    with open(embeddings_file_path, 'w') as embeddings_file:
        json.dump(embeddings_data, embeddings_file, indent=4)

    # Create the nodes.json file (inside the memory_stream folder)
    nodes_data = []  # Empty array for nodes.json
    nodes_file_path = os.path.join(memory_stream_path, "nodes.json")
    with open(nodes_file_path, 'w') as nodes_file:
        json.dump(nodes_data, nodes_file, indent=4)

/var/folders/76/5w1g8h3d1y53y5vcpx55tgl00000gn/T/ipykernel_49771/430785231.py:59: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  scratch_data = {questions[i]: row[i+1] for i in range(len(questions))}


In [ ]:
# go into a directory
# rename all file name from {name}.py tp {name}_memories.py

In [ ]:
"political_ideology": "Progressive",
    "political_party": "Democrat",
    "education": "Bachelor\u2019s Degree in Environmental Studies",
    "race": "African American",
    "ethnicity": "Non-Hispanic",
    "extraversion": 4.5,
    "agreeableness": 4.2,
    "conscientiousness": 3.8,
    "neuroticism": 2.75,
    "openness": 4.7,
    "fact_sheet": "{'childhood': 'Grew up in a middle-class neighborhood, raised by a single mother who was a teacher.', 'education': 'Graduated with honors in Environmental Studies from the University of Alabama.', 'employment': 'Works as an environmental consultant for a nonprofit focused on urban conservation.', 'family': 'No children, close to mother and siblings.', 'romantic_relationships': 'In a long-term relationship with a civil engineer for five years.', 'friendships': 'Close-knit group of friends from college who share her passion for social justice and environmental activism.', 'political_views': 'Strongly identifies with progressive politics and supports Democratic candidates.', 'hobbies': 'Enjoys outdoor activities like hiking, kayaking, and gardening.', 'challenges': 'Struggles with balancing activism and self-care, sometimes experiences burnout.', 'future_plans': 'Aspires to run for local office to advocate for progressive environmental policies.', 'neighborhood': 'Lives in a gentrifying area and is active in organizing for affordable housing.', 'health': 'Leads a healthy lifestyle, practices yoga and maintains a vegetarian diet.', 'religion': 'Raised Christian but identifies as spiritual rather than religious.', 'voting_history': 'Votes in every election and volunteers for voter registration drives.', 'social_media': 'Active on Instagram and Twitter, sharing content on environmental justice and politics.', 'work_life_balance': 'Struggles to disconnect from work but is working on creating better boundaries.'}",
    "speech_pattern": "Speech Pattern: Jasmine has a clear, passionate way of speaking, often using evidence-based arguments to support her views. Her communication style is engaging and thoughtful, particularly when discussing issues that matter to her. Here are the key features of her speech: Logical and Structured: Jasmine speaks with precision and structure, particularly when addressing environmental or political topics. She frequently presents her points with supporting evidence or references to current events. Example: \u201cThe data on climate change is undeniable, and we need local policies that prioritize sustainability.\u201d Empathetic and Inclusive: She often tries to make her speech inclusive, using terms like \u201cwe\u201d and \u201cour community\u201d to emphasize collective responsibility. Example: \u201cWe all have a role to play in protecting our environment.\u201d Motivational Tone: Jasmine is adept at rallying others around her causes, using language that inspires action and change. Example: \u201cNow is the time for all of us to step up. We can\u2019t wait for someone else to make these decisions.\u201d Straightforward but Warm: While she is direct in her communication, especially about difficult topics, she remains approachable and warm, particularly when speaking one-on-one. Example: \u201cI understand how hard this issue can be, but we have to push for progress.\u201d Passionate about Social Justice: When discussing social justice or environmental issues, Jasmine\u2019s tone becomes more animated, and she\u2019s not afraid to express frustration with the status quo. Example: \u201cIt\u2019s frustrating to see that we\u2019re still not addressing racial inequality in environmental policy. We can and should do better.\u201d Occasional Repetition for Emphasis: Jasmine uses repetition for emphasis, particularly when speaking about topics she feels strongly about. Example: \u201cWe can\u2019t wait. We can\u2019t wait any longer for these changes to happen.\u201d Key Examples: Logical and Structured: \u201cThe data on climate change is undeniable, and we need local policies that prioritize sustainability.\u201d Empathetic and Inclusive: \u201cWe all have a role to play in protecting our environment.\u201d Motivational Tone: \u201cNow is the time for all of us to step up. We can\u2019t wait for someone else to make these decisions.\u201d Passionate about Social Justice: \u201cIt\u2019s frustrating to see that we\u2019re still not addressing racial inequality in environmental policy. We can and should do better.\u201d",
    "self_description": "Hi, I\u2019m Jasmine Carter. I was born and raised in Huntsville, Alabama, and I\u2019m passionate about the environment and social justice. My journey began when I was a child, always exploring the outdoors with my grandmother and learning about nature\u2019s importance. That love for the environment led me to study Environmental Studies at the University of Alabama, where I became an advocate for sustainable living and green infrastructure. After college, I started working as an environmental consultant for a nonprofit, and I\u2019ve been dedicated to helping cities develop sustainable solutions ever since. One of the most fulfilling parts of my work is seeing how small changes, like implementing green roofs or increasing access to public transportation, can make such a big difference in urban communities. I\u2019m also deeply committed to social justice, particularly in the context of environmental policy. Too often, marginalized communities bear the brunt of climate change and environmental degradation. I believe it\u2019s my duty to fight for policies that not only protect the environment but also ensure that these communities are prioritized in the solutions. On a personal note, I\u2019m very close to my family, particularly my mom and siblings. I don\u2019t have children of my own yet, but I\u2019m in a long-term relationship with my partner, and we\u2019re happy living our lives focused on our careers and causes for now. I\u2019ve always been someone who tries to live with intention. I\u2019m constantly balancing the demands of my work with self-care, like practicing yoga or spending time in nature. It\u2019s easy to get caught up in the urgency of the issues I care about, but I\u2019m learning the importance of rest and reflection. Looking ahead, I hope to take my advocacy to the next level. I\u2019m considering running for local office in the next few years to directly influence environmental policy in my city. I want to see a world where our communities are healthier, more sustainable, and more just. And I believe we can make it happen, one step at a time.",
    "private_self_description": "Beneath Jasmine\u2019s calm and confident exterior lies a person grappling with the weight of the causes she cares so deeply about. While she comes across as driven and passionate, there are parts of her internal world that remain private: Struggles with Burnout: Jasmine frequently experiences burnout from her advocacy work. While she maintains a public persona of optimism and energy, she sometimes feels overwhelmed by the enormity of the challenges she\u2019s fighting against. The pressure to always \u201cshow up\u201d for her causes can leave her emotionally drained, but she rarely shares this with others for fear of appearing weak or less dedicated. Feelings of Isolation: Despite being surrounded by friends and colleagues who share her values, Jasmine often feels a sense of isolation, especially when it comes to her personal life. She has few close friendships outside of work, and she sometimes wonders if her career and activism are pushing her toward a lonely path. Unresolved Grief: Jasmine lost her grandmother, the person who first instilled in her a love of nature, several years ago. While she channels her grief into her work, she hasn\u2019t fully processed the emotional weight of this loss. She rarely talks about it, preferring to keep her sadness private. Perfectionism and Self-Doubt: Though she appears confident, Jasmine struggles with perfectionism. She sets high standards for herself and often feels she\u2019s not doing enough, even when she\u2019s working tirelessly. This constant self-criticism can be exhausting, and it feeds into her fears of inadequacy, particularly when faced with setbacks in her work. Cynicism about Change: While publicly optimistic about the possibility of creating change, Jasmine privately harbors doubts about whether the system can truly be reformed. She\u2019s seen too many failed initiatives and broken promises, and while she keeps fighting, a part of her worries that meaningful change may never happen in her lifetime. In summary, while Jasmine presents as a driven, passionate individual with a clear sense of purpose, there are deeper layers of emotional complexity, including struggles with burnout, isolation, grief, and self-doubt, that she keeps hidden from those around her. These private struggles shape her internal landscape, making her more human and relatable than her public persona might suggest."


In [44]:
def gpt_predict_profile_1(interview, questions):
    schema = {
        "political_ideology": "<answer>",
        "political_party": "<answer>",
        "education": "<answer>",
        "race":  "<answer>",
        }

    prompt = f"""
    Given the interview transcript and the known information about the person. Answer or predict their political ideology, political_party, education, and race.
    
    --- For example: ---
    Example Input:

  "fact_sheet": 
    "childhood": "Grew up in a middle-class neighborhood, raised by a single mother who was a teacher.",
    "education": "Graduated with honors in Environmental Studies from the University of Alabama.",
    "employment": "Works as an environmental consultant for a nonprofit focused on urban conservation.",
    "family": "No children, close to mother and siblings.",
    "romantic_relationships": "In a long-term relationship with a civil engineer for five years.",
    "friendships": "Close-knit group of friends from college who share her passion for social justice and environmental activism.",
    "political_views": "Strongly identifies with progressive politics and supports Democratic candidates.",
    "hobbies": "Enjoys outdoor activities like hiking, kayaking, and gardening.",
    "challenges": "Struggles with balancing activism and self-care, sometimes experiences burnout.",
    "future_plans": "Aspires to run for local office to advocate for progressive environmental policies.",
    "neighborhood": "Lives in a gentrifying area and is active in organizing for affordable housing.",
    "health": "Leads a healthy lifestyle, practices yoga and maintains a vegetarian diet.",
    "religion": "Raised Christian but identifies as spiritual rather than religious.",
    "voting_history": "Votes in every election and volunteers for voter registration drives.",
    "social_media": "Active on Instagram and Twitter, sharing content on environmental justice and politics.",
    "work_life_balance": "Struggles to disconnect from work but is working on creating better boundaries."
  ,
  "speech_pattern": "Speech Pattern: Jasmine has a clear, passionate way of speaking, often using evidence-based arguments to support her views. Her communication style is engaging and thoughtful, particularly when discussing issues that matter to her. Here are the key features of her speech: Logical and Structured: Jasmine speaks with precision and structure, particularly when addressing environmental or political topics. She frequently presents her points with supporting evidence or references to current events. Example: \u201cThe data on climate change is undeniable, and we need local policies that prioritize sustainability.\u201d Empathetic and Inclusive: She often tries to make her speech inclusive, using terms like \u201cwe\u201d and \u201cour community\u201d to emphasize collective responsibility. Example: \u201cWe all have a role to play in protecting our environment.\u201d Motivational Tone: Jasmine is adept at rallying others around her causes, using language that inspires action and change. Example: \u201cNow is the time for all of us to step up. We can\u2019t wait for someone else to make these decisions.\u201d Straightforward but Warm: While she is direct in her communication, especially about difficult topics, she remains approachable and warm, particularly when speaking one-on-one. Example: \u201cI understand how hard this issue can be, but we have to push for progress.\u201d Passionate about Social Justice: When discussing social justice or environmental issues, Jasmine\u2019s tone becomes more animated, and she\u2019s not afraid to express frustration with the status quo. Example: \u201cIt\u2019s frustrating to see that we\u2019re still not addressing racial inequality in environmental policy. We can and should do better.\u201d Occasional Repetition for Emphasis: Jasmine uses repetition for emphasis, particularly when speaking about topics she feels strongly about. Example: \u201cWe can\u2019t wait. We can\u2019t wait any longer for these changes to happen.\u201d Key Examples: Logical and Structured: \u201cThe data on climate change is undeniable, and we need local policies that prioritize sustainability.\u201d Empathetic and Inclusive: \u201cWe all have a role to play in protecting our environment.\u201d Motivational Tone: \u201cNow is the time for all of us to step up. We can\u2019t wait for someone else to make these decisions.\u201d Passionate about Social Justice: \u201cIt\u2019s frustrating to see that we\u2019re still not addressing racial inequality in environmental policy. We can and should do better.\u201d",
  "self_description": "Hi, I\u2019m Jasmine Carter. I was born and raised in Huntsville, Alabama, and I\u2019m passionate about the environment and social justice. My journey began when I was a child, always exploring the outdoors with my grandmother and learning about nature\u2019s importance. That love for the environment led me to study Environmental Studies at the University of Alabama, where I became an advocate for sustainable living and green infrastructure. After college, I started working as an environmental consultant for a nonprofit, and I\u2019ve been dedicated to helping cities develop sustainable solutions ever since. One of the most fulfilling parts of my work is seeing how small changes, like implementing green roofs or increasing access to public transportation, can make such a big difference in urban communities. I\u2019m also deeply committed to social justice, particularly in the context of environmental policy. Too often, marginalized communities bear the brunt of climate change and environmental degradation. I believe it\u2019s my duty to fight for policies that not only protect the environment but also ensure that these communities are prioritized in the solutions. On a personal note, I\u2019m very close to my family, particularly my mom and siblings. I don\u2019t have children of my own yet, but I\u2019m in a long-term relationship with my partner, and we\u2019re happy living our lives focused on our careers and causes for now. I\u2019ve always been someone who tries to live with intention. I\u2019m constantly balancing the demands of my work with self-care, like practicing yoga or spending time in nature. It\u2019s easy to get caught up in the urgency of the issues I care about, but I\u2019m learning the importance of rest and reflection. Looking ahead, I hope to take my advocacy to the next level. I\u2019m considering running for local office in the next few years to directly influence environmental policy in my city. I want to see a world where our communities are healthier, more sustainable, and more just. And I believe we can make it happen, one step at a time.",
  "private_self_description": "Beneath Jasmine\u2019s calm and confident exterior lies a person grappling with the weight of the causes she cares so deeply about. While she comes across as driven and passionate, there are parts of her internal world that remain private: Struggles with Burnout: Jasmine frequently experiences burnout from her advocacy work. While she maintains a public persona of optimism and energy, she sometimes feels overwhelmed by the enormity of the challenges she\u2019s fighting against. The pressure to always \u201cshow up\u201d for her causes can leave her emotionally drained, but she rarely shares this with others for fear of appearing weak or less dedicated. Feelings of Isolation: Despite being surrounded by friends and colleagues who share her values, Jasmine often feels a sense of isolation, especially when it comes to her personal life. She has few close friendships outside of work, and she sometimes wonders if her career and activism are pushing her toward a lonely path. Unresolved Grief: Jasmine lost her grandmother, the person who first instilled in her a love of nature, several years ago. While she channels her grief into her work, she hasn\u2019t fully processed the emotional weight of this loss. She rarely talks about it, preferring to keep her sadness private. Perfectionism and Self-Doubt: Though she appears confident, Jasmine struggles with perfectionism. She sets high standards for herself and often feels she\u2019s not doing enough, even when she\u2019s working tirelessly. This constant self-criticism can be exhausting, and it feeds into her fears of inadequacy, particularly when faced with setbacks in her work. Cynicism about Change: While publicly optimistic about the possibility of creating change, Jasmine privately harbors doubts about whether the system can truly be reformed. She\u2019s seen too many failed initiatives and broken promises, and while she keeps fighting, a part of her worries that meaningful change may never happen in her lifetime. In summary, while Jasmine presents as a driven, passionate individual with a clear sense of purpose, there are deeper layers of emotional complexity, including struggles with burnout, isolation, grief, and self-doubt, that she keeps hidden from those around her. These private struggles shape her internal landscape, making her more human and relatable than her public persona might suggest."


    Example Output: {{
        "political_ideology": "Progressive",
        "political_party": "Democrat",
        "education": "Bachelor\u2019s Degree in Environmental Studies",
        "race": "African American",
        }}

    ------
    Input: 
    Interview: {interview}
    Q&A: {questions}
    Output format: JSON object matching this structure: {json.dumps(schema, indent=2)}
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": "Provide output in valid JSON with this schema:" + json.dumps(schema)},
            {"role": "user", "content": prompt}
        ],
        max_tokens=300,
        temperature=0.3,
    )
    
    data =  json.loads(response.choices[0].message.content)
    print(data)
    return data

In [45]:
def gpt_predict_profile_2(interview, questions):
    schema = {
        "extraversion": "<number_placeholder>",
        "agreeableness": "<number_placeholder>",
        "conscientiousness":"<number_placeholder>",
        "neuroticism": "<number_placeholder>",
        "openness": "<number_placeholder>",
        }

    prompt = f"""
    You are an experienced counselor. Given the interview transcript and the known information about the person, rate their:

    Extraversion
    Agreeableness
    Conscientiousness
    Neuroticism
    Openness

    on a 1-5 scale, where 1 means not at all and 5 means the maximal manifestation of this quality.
    
    --- For example: ---
    Example Input: 

  "fact_sheet":
    "childhood": "Grew up in a middle-class neighborhood, raised by a single mother who was a teacher.",
    "education": "Graduated with honors in Environmental Studies from the University of Alabama.",
    "employment": "Works as an environmental consultant for a nonprofit focused on urban conservation.",
    "family": "No children, close to mother and siblings.",
    "romantic_relationships": "In a long-term relationship with a civil engineer for five years.",
    "friendships": "Close-knit group of friends from college who share her passion for social justice and environmental activism.",
    "political_views": "Strongly identifies with progressive politics and supports Democratic candidates.",
    "hobbies": "Enjoys outdoor activities like hiking, kayaking, and gardening.",
    "challenges": "Struggles with balancing activism and self-care, sometimes experiences burnout.",
    "future_plans": "Aspires to run for local office to advocate for progressive environmental policies.",
    "neighborhood": "Lives in a gentrifying area and is active in organizing for affordable housing.",
    "health": "Leads a healthy lifestyle, practices yoga and maintains a vegetarian diet.",
    "religion": "Raised Christian but identifies as spiritual rather than religious.",
    "voting_history": "Votes in every election and volunteers for voter registration drives.",
    "social_media": "Active on Instagram and Twitter, sharing content on environmental justice and politics.",
    "work_life_balance": "Struggles to disconnect from work but is working on creating better boundaries."
,
  "speech_pattern": "Speech Pattern: Jasmine has a clear, passionate way of speaking, often using evidence-based arguments to support her views. Her communication style is engaging and thoughtful, particularly when discussing issues that matter to her. Here are the key features of her speech: Logical and Structured: Jasmine speaks with precision and structure, particularly when addressing environmental or political topics. She frequently presents her points with supporting evidence or references to current events. Example: \u201cThe data on climate change is undeniable, and we need local policies that prioritize sustainability.\u201d Empathetic and Inclusive: She often tries to make her speech inclusive, using terms like \u201cwe\u201d and \u201cour community\u201d to emphasize collective responsibility. Example: \u201cWe all have a role to play in protecting our environment.\u201d Motivational Tone: Jasmine is adept at rallying others around her causes, using language that inspires action and change. Example: \u201cNow is the time for all of us to step up. We can\u2019t wait for someone else to make these decisions.\u201d Straightforward but Warm: While she is direct in her communication, especially about difficult topics, she remains approachable and warm, particularly when speaking one-on-one. Example: \u201cI understand how hard this issue can be, but we have to push for progress.\u201d Passionate about Social Justice: When discussing social justice or environmental issues, Jasmine\u2019s tone becomes more animated, and she\u2019s not afraid to express frustration with the status quo. Example: \u201cIt\u2019s frustrating to see that we\u2019re still not addressing racial inequality in environmental policy. We can and should do better.\u201d Occasional Repetition for Emphasis: Jasmine uses repetition for emphasis, particularly when speaking about topics she feels strongly about. Example: \u201cWe can\u2019t wait. We can\u2019t wait any longer for these changes to happen.\u201d Key Examples: Logical and Structured: \u201cThe data on climate change is undeniable, and we need local policies that prioritize sustainability.\u201d Empathetic and Inclusive: \u201cWe all have a role to play in protecting our environment.\u201d Motivational Tone: \u201cNow is the time for all of us to step up. We can\u2019t wait for someone else to make these decisions.\u201d Passionate about Social Justice: \u201cIt\u2019s frustrating to see that we\u2019re still not addressing racial inequality in environmental policy. We can and should do better.\u201d",
  "self_description": "Hi, I\u2019m Jasmine Carter. I was born and raised in Huntsville, Alabama, and I\u2019m passionate about the environment and social justice. My journey began when I was a child, always exploring the outdoors with my grandmother and learning about nature\u2019s importance. That love for the environment led me to study Environmental Studies at the University of Alabama, where I became an advocate for sustainable living and green infrastructure. After college, I started working as an environmental consultant for a nonprofit, and I\u2019ve been dedicated to helping cities develop sustainable solutions ever since. One of the most fulfilling parts of my work is seeing how small changes, like implementing green roofs or increasing access to public transportation, can make such a big difference in urban communities. I\u2019m also deeply committed to social justice, particularly in the context of environmental policy. Too often, marginalized communities bear the brunt of climate change and environmental degradation. I believe it\u2019s my duty to fight for policies that not only protect the environment but also ensure that these communities are prioritized in the solutions. On a personal note, I\u2019m very close to my family, particularly my mom and siblings. I don\u2019t have children of my own yet, but I\u2019m in a long-term relationship with my partner, and we\u2019re happy living our lives focused on our careers and causes for now. I\u2019ve always been someone who tries to live with intention. I\u2019m constantly balancing the demands of my work with self-care, like practicing yoga or spending time in nature. It\u2019s easy to get caught up in the urgency of the issues I care about, but I\u2019m learning the importance of rest and reflection. Looking ahead, I hope to take my advocacy to the next level. I\u2019m considering running for local office in the next few years to directly influence environmental policy in my city. I want to see a world where our communities are healthier, more sustainable, and more just. And I believe we can make it happen, one step at a time.",
  "private_self_description": "Beneath Jasmine\u2019s calm and confident exterior lies a person grappling with the weight of the causes she cares so deeply about. While she comes across as driven and passionate, there are parts of her internal world that remain private: Struggles with Burnout: Jasmine frequently experiences burnout from her advocacy work. While she maintains a public persona of optimism and energy, she sometimes feels overwhelmed by the enormity of the challenges she\u2019s fighting against. The pressure to always \u201cshow up\u201d for her causes can leave her emotionally drained, but she rarely shares this with others for fear of appearing weak or less dedicated. Feelings of Isolation: Despite being surrounded by friends and colleagues who share her values, Jasmine often feels a sense of isolation, especially when it comes to her personal life. She has few close friendships outside of work, and she sometimes wonders if her career and activism are pushing her toward a lonely path. Unresolved Grief: Jasmine lost her grandmother, the person who first instilled in her a love of nature, several years ago. While she channels her grief into her work, she hasn\u2019t fully processed the emotional weight of this loss. She rarely talks about it, preferring to keep her sadness private. Perfectionism and Self-Doubt: Though she appears confident, Jasmine struggles with perfectionism. She sets high standards for herself and often feels she\u2019s not doing enough, even when she\u2019s working tirelessly. This constant self-criticism can be exhausting, and it feeds into her fears of inadequacy, particularly when faced with setbacks in her work. Cynicism about Change: While publicly optimistic about the possibility of creating change, Jasmine privately harbors doubts about whether the system can truly be reformed. She\u2019s seen too many failed initiatives and broken promises, and while she keeps fighting, a part of her worries that meaningful change may never happen in her lifetime. In summary, while Jasmine presents as a driven, passionate individual with a clear sense of purpose, there are deeper layers of emotional complexity, including struggles with burnout, isolation, grief, and self-doubt, that she keeps hidden from those around her. These private struggles shape her internal landscape, making her more human and relatable than her public persona might suggest."



    Example Output: {{
        "extraversion": 4.5,
        "agreeableness": 4.2,
        "conscientiousness": 3.8,
        "neuroticism": 2.75,
        "openness": 4.7,
        }}

    ------
    Input: 
    Interview: {interview}
    Q&A: {questions}
    Output format: JSON object matching this structure: {json.dumps(schema, indent=2)}
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": "Provide output in valid JSON with this schema:" + json.dumps(schema)},
            {"role": "user", "content": prompt}
        ],
        max_tokens=300,
        temperature=0.3,
    )
    
    data =  json.loads(response.choices[0].message.content)
    print(data)
    return data

In [46]:
def gpt_predict_profile_3(interview, questions):
    schema = {
    "fact_sheet": {
        "childhood": "<one_sentence_summary>",
        "education": "<one_sentence_summary>",
        "employment": "<one_sentence_summary>",
        "family": "<one_sentence_summary>",
        "romantic_relationships": "<one_sentence_summary>",
        "friendships": "<one_sentence_summary>",
        "political_views": "<one_sentence_summary>",
        "hobbies": "<one_sentence_summary>",
        "challenges": "<one_sentence_summary>",
        "future_plans": "<one_sentence_summary>",
        "neighborhood": "<one_sentence_summary>",
        "health": "<one_sentence_summary>",
        "religion": "<one_sentence_summary>",
        "voting_history": "<one_sentence_summary>",
        "social_media": "<one_sentence_summary>",
        "work_life_balance": "<one_sentence_summary>"
    },
    "speech_pattern": "<answer_placeholder>",
    "self_description": "<answer_placeholder>",
    "private_self_description": "<answer_placeholder>"
}


    prompt = f"""
    You are a person with following experience and facts. Given these known information, answer about your facts, speech pattern, self description, and private self description. 

    --- For example: ---
    Example Output: 
   {{
  "fact_sheet": {{
    "childhood": "Grew up in a middle-class neighborhood, raised by a single mother who was a teacher.",
    "education": "Graduated with honors in Environmental Studies from the University of Alabama.",
    "employment": "Works as an environmental consultant for a nonprofit focused on urban conservation.",
    "family": "No children, close to mother and siblings.",
    "romantic_relationships": "In a long-term relationship with a civil engineer for five years.",
    "friendships": "Close-knit group of friends from college who share her passion for social justice and environmental activism.",
    "political_views": "Strongly identifies with progressive politics and supports Democratic candidates.",
    "hobbies": "Enjoys outdoor activities like hiking, kayaking, and gardening.",
    "challenges": "Struggles with balancing activism and self-care, sometimes experiences burnout.",
    "future_plans": "Aspires to run for local office to advocate for progressive environmental policies.",
    "neighborhood": "Lives in a gentrifying area and is active in organizing for affordable housing.",
    "health": "Leads a healthy lifestyle, practices yoga and maintains a vegetarian diet.",
    "religion": "Raised Christian but identifies as spiritual rather than religious.",
    "voting_history": "Votes in every election and volunteers for voter registration drives.",
    "social_media": "Active on Instagram and Twitter, sharing content on environmental justice and politics.",
    "work_life_balance": "Struggles to disconnect from work but is working on creating better boundaries."
  }},
  "speech_pattern": "Speech Pattern: Jasmine has a clear, passionate way of speaking, often using evidence-based arguments to support her views. Her communication style is engaging and thoughtful, particularly when discussing issues that matter to her. Here are the key features of her speech: Logical and Structured: Jasmine speaks with precision and structure, particularly when addressing environmental or political topics. She frequently presents her points with supporting evidence or references to current events. Example: \u201cThe data on climate change is undeniable, and we need local policies that prioritize sustainability.\u201d Empathetic and Inclusive: She often tries to make her speech inclusive, using terms like \u201cwe\u201d and \u201cour community\u201d to emphasize collective responsibility. Example: \u201cWe all have a role to play in protecting our environment.\u201d Motivational Tone: Jasmine is adept at rallying others around her causes, using language that inspires action and change. Example: \u201cNow is the time for all of us to step up. We can\u2019t wait for someone else to make these decisions.\u201d Straightforward but Warm: While she is direct in her communication, especially about difficult topics, she remains approachable and warm, particularly when speaking one-on-one. Example: \u201cI understand how hard this issue can be, but we have to push for progress.\u201d Passionate about Social Justice: When discussing social justice or environmental issues, Jasmine\u2019s tone becomes more animated, and she\u2019s not afraid to express frustration with the status quo. Example: \u201cIt\u2019s frustrating to see that we\u2019re still not addressing racial inequality in environmental policy. We can and should do better.\u201d Occasional Repetition for Emphasis: Jasmine uses repetition for emphasis, particularly when speaking about topics she feels strongly about. Example: \u201cWe can\u2019t wait. We can\u2019t wait any longer for these changes to happen.\u201d Key Examples: Logical and Structured: \u201cThe data on climate change is undeniable, and we need local policies that prioritize sustainability.\u201d Empathetic and Inclusive: \u201cWe all have a role to play in protecting our environment.\u201d Motivational Tone: \u201cNow is the time for all of us to step up. We can\u2019t wait for someone else to make these decisions.\u201d Passionate about Social Justice: \u201cIt\u2019s frustrating to see that we\u2019re still not addressing racial inequality in environmental policy. We can and should do better.\u201d",
  "self_description": "Hi, I\u2019m Jasmine Carter. I was born and raised in Huntsville, Alabama, and I\u2019m passionate about the environment and social justice. My journey began when I was a child, always exploring the outdoors with my grandmother and learning about nature\u2019s importance. That love for the environment led me to study Environmental Studies at the University of Alabama, where I became an advocate for sustainable living and green infrastructure. After college, I started working as an environmental consultant for a nonprofit, and I\u2019ve been dedicated to helping cities develop sustainable solutions ever since. One of the most fulfilling parts of my work is seeing how small changes, like implementing green roofs or increasing access to public transportation, can make such a big difference in urban communities. I\u2019m also deeply committed to social justice, particularly in the context of environmental policy. Too often, marginalized communities bear the brunt of climate change and environmental degradation. I believe it\u2019s my duty to fight for policies that not only protect the environment but also ensure that these communities are prioritized in the solutions. On a personal note, I\u2019m very close to my family, particularly my mom and siblings. I don\u2019t have children of my own yet, but I\u2019m in a long-term relationship with my partner, and we\u2019re happy living our lives focused on our careers and causes for now. I\u2019ve always been someone who tries to live with intention. I\u2019m constantly balancing the demands of my work with self-care, like practicing yoga or spending time in nature. It\u2019s easy to get caught up in the urgency of the issues I care about, but I\u2019m learning the importance of rest and reflection. Looking ahead, I hope to take my advocacy to the next level. I\u2019m considering running for local office in the next few years to directly influence environmental policy in my city. I want to see a world where our communities are healthier, more sustainable, and more just. And I believe we can make it happen, one step at a time.",
  "private_self_description": "Beneath Jasmine\u2019s calm and confident exterior lies a person grappling with the weight of the causes she cares so deeply about. While she comes across as driven and passionate, there are parts of her internal world that remain private: Struggles with Burnout: Jasmine frequently experiences burnout from her advocacy work. While she maintains a public persona of optimism and energy, she sometimes feels overwhelmed by the enormity of the challenges she\u2019s fighting against. The pressure to always \u201cshow up\u201d for her causes can leave her emotionally drained, but she rarely shares this with others for fear of appearing weak or less dedicated. Feelings of Isolation: Despite being surrounded by friends and colleagues who share her values, Jasmine often feels a sense of isolation, especially when it comes to her personal life. She has few close friendships outside of work, and she sometimes wonders if her career and activism are pushing her toward a lonely path. Unresolved Grief: Jasmine lost her grandmother, the person who first instilled in her a love of nature, several years ago. While she channels her grief into her work, she hasn\u2019t fully processed the emotional weight of this loss. She rarely talks about it, preferring to keep her sadness private. Perfectionism and Self-Doubt: Though she appears confident, Jasmine struggles with perfectionism. She sets high standards for herself and often feels she\u2019s not doing enough, even when she\u2019s working tirelessly. This constant self-criticism can be exhausting, and it feeds into her fears of inadequacy, particularly when faced with setbacks in her work. Cynicism about Change: While publicly optimistic about the possibility of creating change, Jasmine privately harbors doubts about whether the system can truly be reformed. She\u2019s seen too many failed initiatives and broken promises, and while she keeps fighting, a part of her worries that meaningful change may never happen in her lifetime. In summary, while Jasmine presents as a driven, passionate individual with a clear sense of purpose, there are deeper layers of emotional complexity, including struggles with burnout, isolation, grief, and self-doubt, that she keeps hidden from those around her. These private struggles shape her internal landscape, making her more human and relatable than her public persona might suggest."
}}

    ------
    Input: 
    Interview: {interview}
    Q&A: {questions}
    Output format: JSON object matching this structure: {json.dumps(schema, indent=2)}
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": "Provide output in valid JSON with this schema:" + json.dumps(schema)},
            {"role": "user", "content": prompt}
        ],
        max_tokens=2000,
        temperature=0.2,
    )
    
    data =  json.loads(response.choices[0].message.content)
    print(data)
    return data

In [11]:
df = pd.read_csv("AgentBank/id_to_pseudo.csv")
df.head()

,sunet,pseudo
0,bwthomas,Julian
1,changamy,Sakura
2,riyadule,Riya
3,huanxing,Kai
4,wpo,Jinwoo


In [12]:
# the dataframe has sunet and pesudo two columns

# get all txt file in a folder (currently all named in format {sunet}.txt)

# rename them base on the corresponding pseudo in the data frame


folder_path = 'AgentBank/txt/'  # Set your folder path here
txt_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

for file in txt_files:
    sunet_from_file = file.replace('.txt', '')
    
    pseudo = df.loc[df['sunet'] == sunet_from_file, 'pseudo'].values
    
    if pseudo:
        new_name = f"{pseudo[0]}.txt"  
        old_file_path = os.path.join(folder_path, file)
        new_file_path = os.path.join(folder_path, new_name)
        
        os.rename(old_file_path, new_file_path)
        print(f"Renamed {file} to {new_name}")
    else:
        print(f"Could not find pseudo for {sunet_from_file}")

Renamed bwthomas.txt to Julian.txt
Renamed changamy.txt to Sakura.txt
Renamed riyadule.txt to Riya.txt
Renamed huanxing.txt to Kai.txt
Renamed wpo.txt to Jinwoo.txt
Renamed yinjunw.txt to Mei.txt
Renamed bob9001.txt to Ethan.txt
Renamed jmkrause.txt to Anika.txt
Renamed junyiji3.txt to Jinara.txt
Renamed mlarreaf.txt to Diego.txt
Renamed collinj2.txt to Jinhee.txt
Renamed kjeong.txt to Minji.txt
Renamed micheliu.txt to Mingxia.txt
Renamed nickvo.txt to Anwen.txt
Renamed drewsung.txt to Soojin.txt
Renamed mohnec1.txt to Niran.txt
Renamed aditesh.txt to Arjun.txt
Renamed ealexand.txt to Eleni.txt
Renamed btiryaki.txt to Leyla.txt
Renamed aspark.txt to Jinsoo.txt
Renamed jtian25.txt to Sofia.txt
Renamed bradleyh.txt to Jinaya.txt
Renamed noelsd.txt to Lucia.txt
Renamed tinamai.txt to Meilin.txt
Renamed fraserr.txt to Rafael.txt
Renamed xingyiz.txt to Leila.txt
Renamed vikram29.txt to Anaya.txt
Renamed ruishich.txt to Lianhua.txt
Renamed absinha.txt to Marcus.txt
Renamed kdao.txt to Thao.t

In [ ]:
folder_path = 'AgentBank/txt/'  # Set your folder path here
txt_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

In [16]:
names = df['pseudo'].tolist()
print(len(names))

36


In [54]:
import json
names = df['name'].tolist()
print(names)
for name in names:
    print("-" * 20)
    print("Currently on: ", name)
    interview_file_path = f'AgentBank/txt/{name}.txt'
    questions_file_path = f'charlotte/agent_bank/populations/SyntheticCS222_Base/{name}/scratch.json'

    # Step 1: Get data from the .txt file
    with open(interview_file_path, 'r') as file:
        interview = file.read()  # This is a string from the txt file
    
    # Step 2: Get data from the .json file
    with open(questions_file_path, 'r') as file:
        questions = json.load(file)  # This loads the JSON file as a Python dictionary
    
    demo1 = gpt_predict_profile_1(interview, questions)
    demo2 = gpt_predict_profile_2(interview, questions)
    demo3 = gpt_predict_profile_3(interview, questions)

    merged_dict = {**questions, **demo1, **demo2, **demo3}
    print(merged_dict)

    # go to question_file_path, modify the file to have out (json format)
    with open(questions_file_path, 'w') as file:
        json.dump(merged_dict, file, indent=4)

['Julian', 'Sakura', 'Riya', 'Kai', 'Jinwoo', 'Mei', 'Ethan', 'Jinhee', 'Minji', 'Mingxia', 'Anwen', 'Soojin', 'Arjun', 'Eleni', 'Leyla', 'Jinsoo', 'Sofia', 'Jinaya', 'Lucia', 'Meilin', 'Rafael', 'Leila', 'Anaya', 'Lianhua', 'Marcus', 'Thao', 'Anjali', 'Yuki', 'Marek', 'Kiran']
--------------------
Currently on:  Julian
{'political_ideology': 'Liberal', 'political_party': 'Democrat', 'education': 'Bachelor’s Degree in Economics with a minor in History', 'race': 'White/Caucasian'}
{'extraversion': 3.0, 'agreeableness': 4.0, 'conscientiousness': 4.5, 'neuroticism': 3.5, 'openness': 4.8}
{'fact_sheet': {'childhood': "Julian had a difficult childhood marked by his mother's severe mental illness and his father's struggles with emotional issues.", 'education': "Currently studying economics with a minor in history at Stanford University, with plans to pursue a master's in computer science.", 'employment': 'Julian is focused on his studies and debate competitions, with aspirations to work in r

In [52]:
merged_dict = {**questions, **demo1, **demo2, **demo3}
print(merged_dict)

{'name': 'Julian', 'age group': '18-24', 'gender': 'male', 'grow up location': 'urban', 'Which of the following activities do you spend the most time on?': 'work', 'Which value is most important to you?': 'integrity', 'How would you describe the personality of your closest friend or family member?': 'analytical', 'How do you typically act in unfamiliar social contexts?': 'reserved', 'If you had infinite money, how would you spend most of your time?': 'working on personal projects', 'What is your favorite hobby?': 'reading', 'political affiliation': 'liberal', 'How many places have you lived in?': '4-5', 'What is most important to you in social relationships?': 'intellectual connection', 'How would you describe your childhood?': 'difficult', 'MBTI type': 'entj', 'primary goal for the next 5 years': 'personal development', 'What do you fear the most?': 'rejection', 'Have you experienced any childhood trauma that affects you today?': 'yes', 'How often do you experience intrusive thoughts?

In [56]:
name = "Anaya"
interview_file_path = f'AgentBank/txt/{name}.txt'
questions_file_path = f'charlotte/agent_bank/populations/SyntheticCS222_Base/{name}/scratch.json'

# Step 1: Get data from the .txt file
with open(interview_file_path, 'r') as file:
    interview = file.read()  # This is a string from the txt file
    
# Step 2: Get data from the .json file
with open(questions_file_path, 'r') as file:
    questions = json.load(file)  # This loads the JSON file as a Python dictionary
    
demo1 = gpt_predict_profile_1(interview, questions)
demo2 = gpt_predict_profile_2(interview, questions)
demo3 = gpt_predict_profile_3(interview, questions)

merged_dict = {**questions, **demo1, **demo2, **demo3}
print(merged_dict)

# go to question_file_path, modify the file to have out (json format)
with open(questions_file_path, 'w') as file:
    json.dump(merged_dict, file, indent=4)

{'political_ideology': 'Moderate', 'political_party': 'Independent', 'education': 'Bachelor’s Degree in Computer Science and Applied Mathematics', 'race': 'Asian'}
{'extraversion': 4.5, 'agreeableness': 4.0, 'conscientiousness': 4.2, 'neuroticism': 3.0, 'openness': 4.8}
{'fact_sheet': {'childhood': 'Grew up in a close-knit family in Bombay, surrounded by parents, grandparents, and a brother, with a strong emphasis on discipline and cultural appreciation.', 'education': 'Currently studying Computer Science and Applied Mathematics at Stanford University after a disciplined early education.', 'employment': 'Still exploring career options after graduation, with a focus on personal projects and interests.', 'family': 'Has a supportive family structure that includes parents and a brother, with a strong bond to his grandmother.', 'romantic_relationships': 'Not mentioned in the provided information.', 'friendships': 'Maintains several close friend groups, including high school friends and coll

In [60]:
print(merged_dict)

{'name': 'Anaya', 'age group': '18-24', 'gender': 'male', 'grow up location': 'urban', 'Which of the following activities do you spend the most time on?': 'socializing', 'Which value is most important to you?': 'integrity', 'How would you describe the personality of your closest friend or family member?': 'extroverted', 'How do you typically act in unfamiliar social contexts?': 'confident', 'If you had infinite money, how would you spend most of your time?': 'working on personal projects', 'What is your favorite hobby?': 'arts & crafts', 'political affiliation': 'moderate', 'How many places have you lived in?': '2-3', 'What is most important to you in social relationships?': 'intellectual connection', 'How would you describe your childhood?': 'balanced', 'MBTI type': 'entp', 'primary goal for the next 5 years': 'career growth', 'What do you fear the most?': 'failure', 'Have you experienced any childhood trauma that affects you today?': 'no', 'How often do you experience intrusive thoug

In [61]:
with open(questions_file_path, 'w') as file:
    json.dump(merged_dict, file, indent=4)

In [59]:
print(questions_file_path)

charlotte/agent_bank/populations/SyntheticCS222_Base/Anaya/scratch.json


In [53]:
with open(questions_file_path, 'w') as file:
        json.dump(merged_dict, file, indent=4)